In [1]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [2]:
# Import the cleaned dataset
df_trump = pd.read_csv("parsed_trump.csv")
df_biden = pd.read_csv("parsed_biden.csv")

In [3]:
# Tweet words pre-processing
def preprocess_tweets(tweet):
    tweet_words = []
    
    for word in tweet.split():
        if word.startswith("@") and len(word) > 1:
            word = "@user"
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)
    return " ".join(tweet_words)

In [4]:
# Load the model and the tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ["Negative", "Neutral", "Positive"]

c:\Users\Slez\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Slez\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
c:\Users\Slez\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a ne

In [1]:
df_trump["tweet"] = df_trump["tweet"].apply(preprocess_tweets)

for index, row in df_trump.iterrows():
    encoded_tweet = tokenizer(row['tweet'], return_tensors='pt')
    output = model(**encoded_tweet)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    # You might want to store the results back in the dataframe, for example:
    for i, score in enumerate(scores):
        df_trump.at[index, labels[i]] = score

# This will append the scores for each label as new columns in your DataFrame.

NameError: name 'df_trump' is not defined

In [ ]:
# Save the updated DataFrame to a new Excel file in your drive
df_trump.to_excel('/content/drive/My Drive/saved_tweets_sentiment_analysis.xlsx', index=False)